## Preprocessing

In [3]:
#load data
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("nsmc")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])


Using custom data configuration default
Reusing dataset nsmc (C:\Users\ford0\.cache\huggingface\datasets\nsmc\default\1.1.0\bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


In [4]:
docs[20:50]

['나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고. 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남....바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데 ; 이건 볼게없다 음식도 별로 안나오고, 핀란드 풍경이라도 구경할랫는데 그것도 별로 안나옴 ㅡㅡ',
 '절대 평범한 영화가 아닌 수작이라는걸 말씀드립니다.',
 '주제는 좋은데 중반부터 지루하다',
 '다 짤랐을꺼야. 그래서 납득할 수 없었던거야.. 그럴꺼야.. 꼭 그랬던걸꺼야..',
 'kl2g 고추를 털어버려야 할텐데',
 '카밀라벨 발연기',
 '재밋는뎅',
 '센스있는 연출력..탁월한 캐스팅..90년대의 향수.. 그래서 9점..',
 '엄포스의 위력을 다시 한번 깨닫게 해준 적.남 꽃검사님도 연기 정말 좋았어요! 완전 명품드라마!',
 '졸쓰레기 진부하고말도안됌ㅋㅋ 아..시간아까워',
 '재밌는데 별점이 왜이리 낮은고',
 '1%라도 기대했던 내가 죄인입니다 죄인입니다....',
 '아직도 이 드라마는 내인생의 최고!',
 '패션에 대한 열정! 안나 윈투어!',
 '키이라 나이틀리가 연기하고자 했던건 대체 정신장애일까 틱장애일까',
 '허허...원작가 정신나간 유령이라... 재미있겠네요!',
 '포스터는 있어보이는데 관객은 114명이네',
 '이 영화가 왜 이렇게 저평가 받는지 모르겠다',
 '단순하면서 은은한 매력의 영화',
 "'다 알바생인가 내용도 없고 무서운거도 없고 웃긴거도 하나도 없음 완전 별싱거운 영화.ㅇ.ㅇ내ㅇ시간 넘 아까움 .. . 완전 낚임",
 '오게두어라! 서리한이 굶주렸다!',
 '정말 맘에 들어요. 그래서 또 보고싶은데 또 보는 방법이 없네? >.. ㅜㅡ',
 '윤제문이라는 멋진 배우를 발견하게 됐어요. 소소한 일탈이 잔잔한 미소를 머금게 합니다. 음악은 조금 아쉽네요ㅠㅠ 8점 주고 싶은데 평점 올리고 싶어 10점 줄게요^^',
 '평점에속지마시길시간낭비 돈낭비임',
 '리얼리티

In [10]:
#simplify

import re

docs = dataset['train']['document'] + dataset['test']['document']
label = dataset['train']['label'] + dataset['test']['label']

processed_docs = [re.sub("[\s]+", " ", re.sub("[^가-힣a-zA-Z0-9]", " ", doc)) for doc in docs]

In [6]:
processed_docs[20:50]

['나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남 바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데 이건 볼게없다 음식도 별로 안나오고 핀란드 풍경이라도 구경할랫는데 그것도 별로 안나옴 ',
 '절대 평범한 영화가 아닌 수작이라는걸 말씀드립니다 ',
 '주제는 좋은데 중반부터 지루하다',
 '다 짤랐을꺼야 그래서 납득할 수 없었던거야 그럴꺼야 꼭 그랬던걸꺼야 ',
 'kl2g 고추를 털어버려야 할텐데',
 '카밀라벨 발연기',
 '재밋는뎅',
 '센스있는 연출력 탁월한 캐스팅 90년대의 향수 그래서 9점 ',
 '엄포스의 위력을 다시 한번 깨닫게 해준 적 남 꽃검사님도 연기 정말 좋았어요 완전 명품드라마 ',
 '졸쓰레기 진부하고말도안됌 아 시간아까워',
 '재밌는데 별점이 왜이리 낮은고',
 '1 라도 기대했던 내가 죄인입니다 죄인입니다 ',
 '아직도 이 드라마는 내인생의 최고 ',
 '패션에 대한 열정 안나 윈투어 ',
 '키이라 나이틀리가 연기하고자 했던건 대체 정신장애일까 틱장애일까',
 '허허 원작가 정신나간 유령이라 재미있겠네요 ',
 '포스터는 있어보이는데 관객은 114명이네',
 '이 영화가 왜 이렇게 저평가 받는지 모르겠다',
 '단순하면서 은은한 매력의 영화',
 ' 다 알바생인가 내용도 없고 무서운거도 없고 웃긴거도 하나도 없음 완전 별싱거운 영화 내 시간 넘 아까움 완전 낚임',
 '오게두어라 서리한이 굶주렸다 ',
 '정말 맘에 들어요 그래서 또 보고싶은데 또 보는 방법이 없네 ',
 '윤제문이라는 멋진 배우를 발견하게 됐어요 소소한 일탈이 잔잔한 미소를 머금게 합니다 음악은 조금 아쉽네요 8점 주고 싶은데 평점 올리고 싶어 10점 줄게요 ',
 '평점에속지마시길시간낭비 돈낭비임',
 '리얼리티가 뛰어나긴 한데 큰 공감은 안간다 이민기캐릭터는 정신의학상 분노조절장애 초기 증상일거다 툭하면 사람

In [ ]:
# tokenize

# from konlpy.tag import Mecab
#
# mecab = Mecab()

# res = []

# for doc in docs:
#   tokenlist = mecab.pos(doc)
#   for w in tokenlist:
#     if w[1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP' 'VV', 'VA', 'IC', 'SN', 'SL', 'MAG', 'MAJ']:
#       res.append(w[1])

from konlpy.tag import Okt

okt = Okt()

res = []

for doc in docs:
  tokenlist = mecab.pos(doc)
  temp = []
  for w in tokenlist:
    if w[1] in ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Foreign', 'Alpha', 'Number', 'Unknown']:
      temp.append(w[0])
  res.append(temp)

res[20:50]

In [27]:
import pickle

save = []

for i in range(len(res)):
  save.append(res[i], label[i])

with open("senspos.pickle", 'wb') as f:
  pickle.dump(save, f)


TypeError: append() takes exactly one argument (2 given)

In [33]:
# make v2v
from gensim.models import Word2Vec

model = Word2Vec(res, vector_size=300, window=5, min_count=5, workers=4, sg=0)

In [3]:
# load pretrained w2v
from gensim.models import word2vec

w2v = word2vec.Word2Vec.load("ko.bin")  

In [ ]:
weight = {key: w2v[w] if w in w2v else np.random.uniform(-0.25, 0.25, w2v.vector_size) for k, w in vocabulary_inv.items()}